In [2]:
source("../script/extTADA.R")
source("../script/TADA.R")

Loading required package: ggplot2
Warning message:
: package ‘ggplot2’ was built under R version 3.2.4rstan (Version 2.9.0, packaged: 2016-01-05 16:17:47 UTC, GitRev: 05c3d0058b6a)
For execution on a local, multicore CPU with excess RAM we recommend calling
rstan_options(auto_write = TRUE)
options(mc.cores = parallel::detectCores())

Attaching package: ‘coda’

The following object is masked from ‘package:rstan’:

    traceplot

locfit 1.5-9.1 	 2013-03-22


In [4]:
data <- read.table("../data/data_mut_DD.csv", header = TRUE, as.is = TRUE)
head(data)

,Gene,mut_lof,mut_missense,mut_damaging,dn_damaging_DD,dn_lof_DD,dn_missense_DD,dn_silent_DD
1,A1BG,9.996657e-07,2.300224e-05,4.54062e-07,0,0,0,0
2,A1BG-AS1,1.420491e-07,0,0,0,0,0,0
3,A1CF,2.144318e-06,1.68827e-05,3.35199e-06,0,0,0,0
4,A2M,3.981797e-06,4.039597e-05,7.82154e-06,0,0,0,0
5,A2M-AS1,9.14698e-08,0,0,0,0,0,0
6,A2ML1,3.779929e-06,4.007178e-05,2.85364e-06,0,0,1,0


In [5]:
t1 <- as.numeric(data$mut_lof)
t1 <- t1[t1 > 0]
data[data$mut_lof == 0, ]$mut_lof <- min(t1)/10
#data <- data[data$mut_missense >0, ]
t2 <- as.numeric(data$mut_missense)
t2 <- t2[t2 > 0]
data[data$mut_missense == 0, ]$mut_missense <- min(t2)/10
dim(data)

#data <- data[data$mut_missense >0, ]
t2 <- as.numeric(data$mut_damaging)
t2 <- t2[t2 > 0]
data[data$mut_damaging == 0, ]$mut_damaging <- min(t2)/10

[1] 19358     8

In [7]:
allDNData <- data[, paste0("dn_", c("damaging", "lof"), "_DD")]
allMutData <- data[,paste0("mut_", c("damaging", "lof"))]
head(data.frame(allMutData, allDNData))


,mut_damaging,mut_lof,dn_damaging_DD,dn_lof_DD
1,4.54062e-07,9.996657e-07,0,0
2,1.04146e-10,1.420491e-07,0,0
3,3.35199e-06,2.144318e-06,0,0
4,7.82154e-06,3.981797e-06,0,0
5,1.04146e-10,9.14698e-08,0,0
6,2.85364e-06,3.779929e-06,0,0


In [21]:
ntrioDD = 4293
modelDNdata <- list(NN = dim(allDNData)[1], #Gene numbers
                    K = 2, #Hypothesis numbers: should be default
                    NCdn = 2, #Number of de novo classes
                    Ndn = ntrioDD, # Family numbers
                    dataDN = data.frame(allDNData), # Denovo data
                    mutRate = data.frame(allMutData), # Mutation rates 
                    betaPars = c(6.7771073, -1.7950864, -0.2168248), #Adjust beta's values: should be default
                    adjustHyperBeta = as.integer(0), ##1 if want to adjust beta, 0 if don't want to adjust beta
                    upperPi0 = 0.5, lowerPi0 = 0, ##Lower and upper limits of pi: should be default
                    lowerHyperGamma = 1, lowerGamma = 1, #Should be default 
                    hyperBetaDN0 = array(c(5, 1))) ##Default beta values

In [22]:
nIteration = 1000
nChain = 1
nCore = 1
nThin = 1
mcmcDD <- stan(model_code = DNextTADA, ##Use only De novo model inside extTADA
               data = modelDNdata, ##Model data as described above
               iter = nIteration, ## 
               chains = nChain, 
               cores = nCore,
               thin = nThin)

SYNTAX ERROR, MESSAGE(S) FROM PARSER:

Illegal statement beginning with non-void expression parsed as
  hyperBetaDN[i2i]
Not a legal assignment, sampling, or function statement.  Note that
  * Assignment statements only allow variables (with optional indexes) on the left;
    if you see an outer function logical_lt (<) with negated (-) second argument,
    it indicates an assignment statement A <- B with illegal left
    side A parsed as expression (A < (-B)).
  * Sampling statements allow arbitrary value-denoting expressions on the left.
  * Functions used as statements must be declared to have void returns


ERROR at line 34

 32:        if (adjustHyperBeta != 0) {
 33:          for (i2i in 1:NCdn){
 34:                hyperBetaDN[i2i] = exp(betaPars[1]*hyperGammaMeanDN[i2i]^(betaPars[2]) + betaPars[3]); 
                    ^
 35:          

PARSER EXPECTED: "}"


ERROR: Error in stanc(file = file, model_code = model_code, model_name = model_name, : failed to parse Stan model 'dd55ce02a9af35e60451a5abf7ad1392' due to the above error.
